# Teste de Conexão dos modulos de extração

## SQL Server

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # Adiciona o diretório pai ao PYTHONPATH
from scripts.extract.exportar_dados_para_parquet import salvar_dataframe_em_parquet

In [2]:
query = '''
	SELECT DISTINCT
        SPL.CODPERLET,
        ISNULL(SHF.CODGRADE,STH.CODGRADE) AS CODGRADE,
        CASE
            WHEN SHF.CODGRADE = '0301' OR STH.CODGRADE = '0301' THEN '1º Ano'
            WHEN SHF.CODGRADE = '0302' OR STH.CODGRADE = '0302' THEN '2º Ano'
            WHEN SHF.CODGRADE = '0303' OR STH.CODGRADE = '0303' THEN '3º Ano'
            END AS SERIE,
            STD.CODTURMA,
        ET.DESCRICAO AS DESC_ETAPA,
        ETP.LABEL,
        STD.CODDISC,
        SD.NOME AS DISCIPLINA,
		CASE	
			WHEN STD.CODDISC LIKE '%.I' THEN 'Interdisciplinar'
			WHEN STD.CODDISC LIKE '%.E' THEN 'Eletiva'
			WHEN (STD.CODDISC = 'ING' OR STD.CODDISC = 'ESP') THEN 'Turma Mista'
			ELSE 'Regular'
			END AS TIPO_DISCIPLINA,
        AV.AVALIACAO ID_AVALIACAO,
        AV.NOME AS AVALIACAO,
        AV.PUBLICAR,
        AV.PESO,
        TA.DESCRICAO AS TIPO_AVALIACAO,
        TE.NOME AS ETAPA,
        ATD.DATA AS DATA_PROVA,
        --AV.DATAHORAREC AS DATA_CRIACAO_AVALIACAO,
        		    CASE 
        WHEN av.ULTIMOPERIODO = 1 THEN 'Sim'
        ELSE 'Não'
    END AS ULTIMOPERIODO_STATUS,
        GETDATE() AS 'DATA_ULT_ATUALIZACAO_DADOS'
    FROM
        AVALIACOES AV
        INNER JOIN AVALIACOESTURMASDISC ATD ON ATD.AVALIACAO = AV.AVALIACAO
        INNER JOIN ETAPAS ET ON ET.ETAPA = AV.ETAPA
        INNER JOIN ETAPAS ETP ON ETP.ETAPA = ET.ETAPAPAI
        INNER JOIN TIPOSETAPAS TE ON TE.TIPOETAPA = ETP.TIPOETAPA
        INNER JOIN TIPOSAVALIACOES TA ON TA.TIPO = AV.TIPO
        INNER JOIN CorporeRM.dbo.STURMADISC STD ON STD.CODCOLIGADA = ATD.CODCOLIGADA AND STD.IDTURMADISC = ATD.IDTURMADISC
        LEFT JOIN CorporeRM.dbo.SHABILITACAOFILIAL SHF ON SHF.CODCOLIGADA = STD.CODCOLIGADA AND SHF.IDHABILITACAOFILIAL = STD.IDHABILITACAOFILIAL 
        INNER JOIN CorporeRM.dbo.SDISCIPLINA SD ON SD.CODCOLIGADA = STD.CODCOLIGADA AND SD.CODDISC = STD.CODDISC
        INNER JOIN CorporeRM.dbo.SPLETIVO SPL ON SPL.CODCOLIGADA = STD.CODCOLIGADA AND SPL.IDPERLET = STD.IDPERLET
        INNER JOIN CorporeRM.dbo.STURMADISCHABILITACAOPL STH ON STH.CODCOLIGADA = STD.CODCOLIGADA AND STH.CODFILIAL = STD.CODFILIAL AND STH.IDTURMADISC = STD.IDTURMADISC
    WHERE
        STD.CODCOLIGADA = 2         /* COLIGADA DO ENSINO MÉDIO */
        AND SPL.CODPERLET >= 2023  /* ANO LETIVO */
        AND ETP.TIPOETAPA = 1       /* REGULAR */
        AND STD.CODDISC NOT LIKE '%.C'
        AND AV.EXIBEBI = 1
    ORDER BY
        CODGRADE,
        LABEL,
        DISCIPLINA,
        ATD.DATA;
        '''

In [3]:
credenciais_sqlserver = {
    "server": "192.168.16.204",
    "user": "app_bi",
    "password": "4BEA24B6FBDB495C"
}

In [4]:
# Exemplo de uso
salvar_dataframe_em_parquet(
    query=query,
    nome_arquivo="teste",
    credenciais=credenciais_sqlserver,
    nome_banco="diarioMB",
    tipo_banco="sqlserver",
    path_file="../data/raw/parquet"
)

/home/leandric/Documents/DataBridgeX2/scripts/extract/exportar_dados_para_parquet.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DataFrame salvo com sucesso em: ../data/raw/parquet/diarioMB.teste.parquet


In [9]:
import pandas as pd
df = pd.read_parquet("../data/raw/parquet/diarioMB.teste.parquet")
df

,CODPERLET,CODGRADE,SERIE,CODTURMA,DESC_ETAPA,LABEL,CODDISC,DISCIPLINA,TIPO_DISCIPLINA,ID_AVALIACAO,AVALIACAO,PUBLICAR,PESO,TIPO_AVALIACAO,ETAPA,DATA_PROVA,ULTIMOPERIODO_STATUS,DATA_ULT_ATUALIZACAO_DADOS
0,2024,0301,1º Ano,1ABE1,1º Semestre - 1º Período - Médio,1º Semestre,ABE.E,Aspectos básicos da economia,Eletiva,107116,Sondagem 1,1,25.0,SONDAGEM,REGULAR,2024-03-20,Não,2025-01-24 15:53:09.907
1,2024,0301,1º Ano,1ABE2,1º Semestre - 1º Período - Médio,1º Semestre,ABE.E,Aspectos básicos da economia,Eletiva,107116,Sondagem 1,1,25.0,SONDAGEM,REGULAR,2024-03-20,Não,2025-01-24 15:53:09.907
2,2024,0301,1º Ano,1ABE1,1º Semestre - 2º Período - Médio,1º Semestre,ABE.E,Aspectos básicos da economia,Eletiva,107930,Sondagem 2,1,10.0,SONDAGEM,REGULAR,2024-04-23,Não,2025-01-24 15:53:09.907
3,2024,0301,1º Ano,1ABE2,1º Semestre - 2º Período - Médio,1º Semestre,ABE.E,Aspectos básicos da economia,Eletiva,107930,Sondagem 2,1,10.0,SONDAGEM,REGULAR,2024-04-23,Não,2025-01-24 15:53:09.907
4,2024,0301,1º Ano,1ABE1,1º Semestre - 2º Período - Médio,1º Semestre,ABE.E,Aspectos básicos da economia,Eletiva,108220,Cadernos de pesquisa,1,20.0,SONDAGEM,REGULAR,2024-05-13,Não,2025-01-24 15:53:09.907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10316,2024,0303,3º Ano,C3FM,2º Semestre - 6º Período - Médio,2º Semestre,UNI,Unificada,Regular,113469,PR6 - 3º ANO - LP/GEO,0,0.0,P - TESTE,REGULAR,2024-10-22,Não,2025-01-24 15:53:09.907
10317,2024,0303,3º Ano,C3FM,2º Semestre - 6º Período - Médio,2º Semestre,UNI,Unificada,Regular,113471,PR6 - 3º ANO - FÍS/HIS,0,0.0,P - TESTE,REGULAR,2024-10-22,Não,2025-01-24 15:53:09.907
10318,2024,0303,3º Ano,C3FM,2º Semestre - 6º Período - Médio,2º Semestre,UNI,Unificada,Regular,113472,PR6 - 3º ANO - BIO/EC,0,0.0,P - TESTE,REGULAR,2024-10-22,Não,2025-01-24 15:53:09.907
10319,2024,0303,3º Ano,C3FM,2º Semestre - 6º Período - Médio,2º Semestre,UNI,Unificada,Regular,113474,PR6 - 3º ANO - QUÍ/EL,0,0.0,P - TESTE,REGULAR,2024-10-22,Não,2025-01-24 15:53:09.907


## MySQL

In [12]:
query = '''
SELECT
	STU.identification AS 'ALUNO_RA',
    STU.name AS 'ALUNO_NOME',
    SQ.class_code AS 'ALUNO_CLASSE',	-- PEGANDO A TURMA DA LIGAÇÃO DE ESTUDANTES E QUIZZES
    LEFT(QUI.grade,1) AS 'ALUNO_GRADE', -- PEGANDO GRADE DA PROVA
    SQ.identification AS 'DIARIO_AVALIACOESALUNOS',
    SQ.final_score AS 'NOTA_FINAL',
    QUI.id AS 'PROVA_ID',
    QUI.name AS 'PROVA_NOME',
    QUI.question_quantity AS 'PROVA_QTD_QUESTOES',
    QUI.year AS 'ANO_LETIVO',
    QUI.taken_date AS 'PROVA_DATA',
    QUI.status AS 'PROVA_STATUS',
    QUI.stage AS 'PROVA_PERIODO',
    QUI.maximum_points AS 'PROVA_NOTA_MAX',
    SUB.name AS 'DISCIPLINA',
    SUB.code AS 'DISCIPLINA _COD', 
    QQ.question_number AS 'QUESTAO_NUM',
    QQ.maximum_points AS 'QUESTAO_NOTA_MAX',
    AP.question_penalty AS 'QUESTAO_PONTUACAO',
    STA.id AS 'CARIMBO_ID',
    STA.name AS 'CARIMBO_NOME',
    STA.description AS 'CARIMBO_DESC',
    CAT.name AS 'CAT_NOME',
    CAT.color AS 'CAT_COLOR',
    NOW() AS 'ULTIMA_ATUALIZACAO',
    CASE	
		WHEN SUB.code LIKE '%.I' THEN 'Interdisciplinar'
		WHEN SUB.code LIKE '%.E' THEN 'Eletiva'
		WHEN (SUB.code = 'ING' OR SUB.code = 'ESP') THEN 'Turma Mista'
		ELSE 'Regular'
		END AS TIPO_DISCIPLINA,
	SF.folder_id AS 'PASTA_ID',
    F.name AS 'PASTA'
FROM
	students STU
    INNER JOIN student_quizzes SQ ON SQ.student_id = STU.id
    INNER JOIN quizzes QUI ON QUI.id = SQ.quiz_id
    INNER JOIN quiz_questions QQ ON QQ.quiz_id = QUI.id
    INNER JOIN questions QUE ON QUE.quiz_question_id = QQ.id AND QUE.student_quiz_id = SQ.id
    LEFT JOIN (SELECT
					APS.id,
					APS.question_id,
					APS.stampable_type,
					APS.stampable_id,
					CASE
						WHEN APS.stampable_type = 'QuizStamp' THEN QUIS.question_penalty
						WHEN APS.stampable_type = 'QuestionStamp' THEN QUES.question_penalty
						END AS 'question_penalty',
					CASE
						WHEN APS.stampable_type = 'QuizStamp' THEN QUIS.stamp_id
						WHEN APS.stampable_type = 'QuestionStamp' THEN QUES.stamp_id
						END AS 'stamp_id'
				FROM
					applied_stamps APS
					LEFT JOIN quiz_stamps QUIS ON QUIS.id = APS.stampable_id AND APS.stampable_type = 'QuizStamp'
					LEFT JOIN question_stamps QUES ON QUES.id = APS.stampable_id AND APS.stampable_type = 'QuestionStamp'
				) AP ON AP.question_id = QUE.id
    INNER JOIN stamps STA ON STA.id = AP.stamp_id
    INNER JOIN stamp_folders SF ON SF.stamp_id = STA.id
    INNER JOIN folders F ON F.id = SF.folder_id
    INNER JOIN categories CAT ON CAT.id = STA.category_id
    INNER JOIN subjects SUB ON SUB.id = QUI.subject_id
WHERE
	QUI.year >= 2022
   -- AND STU.identification = '100045'		-- RA
   -- AND SQ.identification = '6630192'		-- AVALIAÇÕES ALUNOS
   -- AND QUI.id = '11' 					-- ID DA PROVA
ORDER BY
	STU.name,
    QUI.id,
    QQ.question_number,
    STA.name
-- LIMIT 100
;
'''

In [14]:
credenciais_mysql = {
    "host": "192.168.18.65",
    "user": "data.etl",
    "password": "ooL!09)$_llsoPPL0",
    "port": 3306
}

In [15]:
salvar_dataframe_em_parquet(
    query=query,
    nome_arquivo="testemysql",
    credenciais=credenciais_mysql,
    nome_banco="escola_mobile_corretor",
    tipo_banco="mysql",
    path_file="../data/raw/parquet"
)

DataFrame salvo com sucesso em: ../data/raw/parquet/escola_mobile_corretor.testemysql.parquet


In [17]:
df = pd.read_parquet("../data/raw/parquet/escola_mobile_corretor.testemysql.parquet")
df

,ALUNO_RA,ALUNO_NOME,ALUNO_CLASSE,ALUNO_GRADE,DIARIO_AVALIACOESALUNOS,NOTA_FINAL,PROVA_ID,PROVA_NOME,PROVA_QTD_QUESTOES,ANO_LETIVO,...,QUESTAO_PONTUACAO,CARIMBO_ID,CARIMBO_NOME,CARIMBO_DESC,CAT_NOME,CAT_COLOR,ULTIMA_ATUALIZACAO,TIPO_DISCIPLINA,PASTA_ID,PASTA
0,100045,ADRIANA KIAN WANDALSEN,F,1,6923919,4.1,76,PR1 DISCURSIVA,5,2023,...,0.4,2643,Explicação correta,"Explicação correta e completa, sem imprecisões...",Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,71,Biologia - 1º Ano - 2023
1,100045,ADRIANA KIAN WANDALSEN,F,1,6923919,4.1,76,PR1 DISCURSIVA,5,2023,...,0.0,2648,Explicação incorreta,Explicação incorreta.,Descontos,E55A72,2025-01-24 19:16:18,Regular,71,Biologia - 1º Ano - 2023
2,100045,ADRIANA KIAN WANDALSEN,F,1,6923919,4.1,76,PR1 DISCURSIVA,5,2023,...,0.2,2576,Relação ecológica,Identifica corretamente a relação ecológica.,Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,71,Biologia - 1º Ano - 2023
3,100045,ADRIANA KIAN WANDALSEN,F,1,6923919,4.1,76,PR1 DISCURSIVA,5,2023,...,0.2,2576,Relação ecológica,Identifica corretamente a relação ecológica.,Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,41,Biologia
4,100045,ADRIANA KIAN WANDALSEN,F,1,6923919,4.1,76,PR1 DISCURSIVA,5,2023,...,0.3,1216,Resposta correta,"Resposta correta e justificativa coerente, sem...",Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,71,Biologia - 1º Ano - 2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030546,181444,ZIXIN RU,A,1,8327783,7.9,525,PR5 DISSERTATIVA,5,2024,...,1.0,16694,Resolução parcialmente correta,Resolução parcialmente correta (analisar desco...,Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,212,Biologia 1º ano - 2024
1030547,181444,ZIXIN RU,A,1,8327783,7.9,525,PR5 DISSERTATIVA,5,2024,...,0.0,9632,Resposta incorreta,Resposta incorreta,Descontos,E55A72,2025-01-24 19:16:18,Regular,212,Biologia 1º ano - 2024
1030548,181444,ZIXIN RU,A,1,8327783,7.9,525,PR5 DISSERTATIVA,5,2024,...,1.0,16693,Resolução correta,Resolução correta,Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,212,Biologia 1º ano - 2024
1030549,181444,ZIXIN RU,A,1,8327783,7.9,525,PR5 DISSERTATIVA,5,2024,...,0.5,16975,Resolução correta.,Resolução correta.,Atribuição de pontuação,4EB857,2025-01-24 19:16:18,Regular,212,Biologia 1º ano - 2024
